<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Example</a></span></li><li><span><a href="#Cleaning" data-toc-modified-id="Cleaning-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Cleaning</a></span></li><li><span><a href="#Final-Advice" data-toc-modified-id="Final-Advice-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Final Advice</a></span></li></ul></li><li><span><a href="#Data-cleaning-(wrangling)" data-toc-modified-id="Data-cleaning-(wrangling)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data cleaning (wrangling)</a></span><ul class="toc-item"><li><span><a href="#portfolio:-Offers-sent-during-30-day-test-period-(10-offers-x-6-fields)" data-toc-modified-id="portfolio:-Offers-sent-during-30-day-test-period-(10-offers-x-6-fields)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>portfolio: Offers sent during 30-day test period (10 offers x 6 fields)</a></span></li><li><span><a href="#profile:-Rewards-program-users-(17000-users-x-5-fields)" data-toc-modified-id="profile:-Rewards-program-users-(17000-users-x-5-fields)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>profile: Rewards program users (17000 users x 5 fields)</a></span></li><li><span><a href="#transcript:-Event-log-(306648-events-x-4-fields)" data-toc-modified-id="transcript:-Event-log-(306648-events-x-4-fields)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>transcript: Event log (306648 events x 4 fields)</a></span></li><li><span><a href="#Merge-datasets" data-toc-modified-id="Merge-datasets-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Merge datasets</a></span></li></ul></li></ul></div>

# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:


**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record



In [1]:
import pandas as pd
import numpy as np
import math
import json
import os
%matplotlib inline

from pathlib import Path
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Starbucks'

In [2]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Starbucks/Data')

In [3]:
# read in the json files
portfolio = pd.read_json(path/'portfolio.json', orient='records', lines=True)
profile = pd.read_json(path/'profile.json', orient='records', lines=True)
transcript = pd.read_json(path/'transcript.json', orient='records', lines=True)

In [4]:
portfolio.shape, transcript.shape, profile.shape

((10, 6), (306534, 4), (17000, 5))

## Data cleaning (wrangling)

### portfolio: Offers sent during 30-day test period (10 offers x 6 fields)

- reward: (numeric) money awarded for the amount spent
- channels: (list) web, email, mobile, social
- difficulty: (numeric) minimum required spend to complete an offer
- duration: (numeric) time for offer to be open, in days
- offer_type: (string) bogo, discount, informational
- id: (string/hash) __offer id__


__Need to clean__
- `portfolio['channels']` have multiple information in each row. We can make each channel's columns and input boolean values (0, 1). 

- `portfolio['id']`'column name change to 'offer_id' for convenience when we merge the datasets. 



In [5]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [6]:
portfolio = portfolio.rename(columns={'id': 'offer_id'})

In [7]:
# create a column which indicates offer id 
portfolio['offer_number'] = ['offer_' + str(portfolio.index[i]) for i in range(10)]

# create columns which has each channel's info with boolean values (0,1) 
portfolio['email'] = [1 if 'email' in portfolio['channels'][i] else 0 for i in range(10)]
portfolio['mobile'] = [1 if 'mobile' in portfolio['channels'][i] else 0 for i in range(10)]
portfolio['social'] = [1 if 'social' in portfolio['channels'][i] else 0 for i in range(10)]
portfolio['web'] = [1 if 'web' in portfolio['channels'][i] else 0 for i in range(10)]

portfolio.head()

,reward,channels,difficulty,duration,offer_type,offer_id,offer_number,email,mobile,social,web
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,offer_0,1,1,1,0
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,offer_1,1,1,1,1
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,offer_2,1,1,0,1
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_3,1,1,0,1
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_4,1,0,0,1


In [8]:
# drop channels column
portfolio.drop('channels', axis=1, inplace=True)
portfolio

,reward,difficulty,duration,offer_type,offer_id,offer_number,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,offer_0,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,offer_1,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,offer_2,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_3,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_4,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,offer_5,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,offer_6,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,offer_7,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,offer_8,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,offer_9,1,1,0,1


### profile: Rewards program users (17000 users x 5 fields)

- gender: (categorical) M, F, O, or null (note some entries contain 'O' for other rather than M or F)
- age: (numeric) missing value encoded as 118
- id: (string/hash) customer id
- became_member_on: (date) format YYYYMMDD: date when customer created an app account
- income: (numeric) customer's income





__Need to clean:__
- change `id` column name to `customer_id`

- `profile['became_member_on']` should be datetime data. 
- Create `year`, `month`, and `day` columns by extracting from `became_member_on` column.

- Change `profile['age']` values which have 118 value to 'NaN' 





In [9]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [10]:
profile = profile.rename(columns={'id': 'customer_id'})

In [11]:
profile['became_date'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['year'] = profile['became_date'].dt.year
profile['month'] = profile['became_date'].dt.month
profile['day'] = profile['became_date'].dt.day

profile.head()

,gender,age,customer_id,became_member_on,income,became_date,year,month,day
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,2017-02-12,2017,2,12
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,2017-07-15,2017,7,15
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,2018-07-12,2018,7,12
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,2017-05-09,2017,5,9
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,2017-08-04,2017,8,4


In [12]:
# check if there are missing values
profile.isnull().sum()

gender              2175
age                    0
customer_id            0
became_member_on       0
income              2175
became_date            0
year                   0
month                  0
day                    0
dtype: int64

In [13]:
profile.shape

(17000, 9)

Noted that customers who don't have their gender information in dataset also don't have income and age information neither. 

In [14]:
assert list(profile[profile['gender']=='None'].index) == list(profile[profile['income']=='NaN'].index)

In [15]:
# change profile['age']==118 values to NaN
profile['age'] = profile['age'].replace(118, 'NaN')
profile.head()

,gender,age,customer_id,became_member_on,income,became_date,year,month,day
0,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,2017-02-12,2017,2,12
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,2017-07-15,2017,7,15
2,None,NaN,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,2018-07-12,2018,7,12
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,2017-05-09,2017,5,9
4,None,NaN,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,2017-08-04,2017,8,4


In [16]:
# drop became_member_on
profile.drop('became_member_on', axis=1, inplace=True)
profile.head()

,gender,age,customer_id,income,became_date,year,month,day
0,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,2017,7,15
2,None,NaN,38fe809add3b4fcf9315a9694bb96ff5,NaN,2018-07-12,2018,7,12
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09,2017,5,9
4,None,NaN,a03223e636434f42ac4c3df47e8bac43,NaN,2017-08-04,2017,8,4


### transcript: Event log (306648 events x 4 fields)

- person: (string/hash) customer id
- event: (string) record description (offer received, offer viewed, transaction, offer completed)
- value: (dictionary) different values depending on event type
    - offer id: (string/hash) not associated with any "transaction"
    - amount: (numeric) money spent in "transaction"
    - reward: (numeric) money gained from "offer completed"
- time: (numeric) hours after start of test


__Need to clean:__

- Change `person` column name to `customer_id`
- Create columns for different event type values - offer_id, amount, and reward by extracting inf from `value` column. 

    

In [17]:
transcript = transcript.rename(columns={'person': 'customer_id'})

transcript['value'] has a combination of 'id', 'offer id', 'reward' and 'amount'. 

In [18]:
def split_values(all_values):
    """ Function to get each value depending on event type from transcript.value column, 
    and create each column of offer_id, reward, and amount that has boolean values (0,1)"""
    # initialize lists
    value_lists = [[], [], []]
    for val in all_values:
        keys = val.keys()
        
        # getting the 'offer id' and 'offer_id'.
        if 'offer id' in keys: value_lists[0].append(val['offer id'])
        else:
            if 'offer_id' in keys: value_lists[0].append(val['offer_id'])
            else: value_lists[0].append(np.nan)
                
        # getting 'reward'
        if 'reward' in keys: value_lists[1].append(val['reward'])
        else:
            value_lists[1].append(np.nan)
            
        # getting 'amount'
        if 'amount' in keys: value_lists[2].append(val['amount'])
        else:
            value_lists[2].append(np.nan)
            
        lst_offer = value_lists[0]
        lst_reward = value_lists[1]
        lst_amount = value_lists[2]
        
    return lst_offer, lst_reward, lst_amount

In [19]:
# unzip the columns value to get {offer_id, reward, amount}
transcript['offer_id'], transcript['reward'], transcript['amount'] = split_values(transcript['value'])

In [20]:
transcript.head()

,customer_id,event,value,time,offer_id,reward,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


There are four types of events: __'offer received', 'offer viewed', 'transaction', 'offer completed'__

- When event shows "__offer received__", the value has 'offer id'
- When event shows "__offer viewed__", the value has 'offer id'
- When event shows "__transaction__", the value has 'amount'
- When event shows "__offer completed__", the value has 'offer id' and 'reward'

In [21]:
# drop value column
transcript.drop('value', axis=1, inplace=True)

### Merge datasets

Let's merge three datasets

In [22]:
profile = profile.merge(transcript, how='left', on='customer_id')
df = profile.merge(portfolio, how='left', on='offer_id')
df.head()

,gender,age,customer_id,income,became_date,year,month,day,event,time,...,amount,reward_y,difficulty,duration,offer_type,offer_number,email,mobile,social,web
0,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12,offer received,168,...,NaN,2.0,10.0,7.0,discount,offer_9,1.0,1.0,0.0,1.0
1,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12,offer viewed,216,...,NaN,2.0,10.0,7.0,discount,offer_9,1.0,1.0,0.0,1.0
2,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12,offer received,336,...,NaN,5.0,20.0,10.0,discount,offer_4,1.0,0.0,0.0,1.0
3,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12,offer viewed,348,...,NaN,5.0,20.0,10.0,discount,offer_4,1.0,0.0,0.0,1.0
4,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017,2,12,transaction,360,...,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.shape

(306534, 22)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306534 entries, 0 to 306533
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   gender        272762 non-null  object        
 1   age           306534 non-null  object        
 2   customer_id   306534 non-null  object        
 3   income        272762 non-null  float64       
 4   became_date   306534 non-null  datetime64[ns]
 5   year          306534 non-null  int64         
 6   month         306534 non-null  int64         
 7   day           306534 non-null  int64         
 8   event         306534 non-null  object        
 9   time          306534 non-null  int64         
 10  offer_id      167581 non-null  object        
 11  reward_x      33579 non-null   float64       
 12  amount        138953 non-null  float64       
 13  reward_y      167581 non-null  float64       
 14  difficulty    167581 non-null  float64       
 15  duration      167